In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from collections import Counter

# Mapping จากชื่อคลาสไปยัง index
classes = ('161', '162', '163', '164', '167', '168', '169', '170', '171', '173', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195')

class_to_idx = {'161': 0, '162': 1, '163': 2, '164': 3, '167': 4, '168': 5, '169': 6, '170': 7, '171': 8, '173': 9, 
                '175': 10, '176': 11, '177': 12, '178': 13, '179': 14, '180': 15, '181': 16, '182': 17, '183': 18, 
                '184': 19, '185': 20, '186': 21, '187': 22, '188': 23, '189': 24, '190': 25, '191': 26, '192': 27, 
                '193': 28, '194': 29, '195': 30}

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, threshold=500, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform if transform else transforms.ToTensor()
        self.target_transform = target_transform

        # นับจำนวนตัวอย่างในแต่ละคลาสโดยใช้ collections.Counter
        self.class_sample_counts = Counter(self.img_labels.iloc[:, 1])
        self.threshold = threshold

        # การทำ Data Augmentation สำหรับคลาสที่มีตัวอย่างน้อย
        self.augment_transform = transforms.Compose([
            transforms.RandomRotation(degrees=10),
            transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.2)),
            transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), shear=10),
            transforms.GaussianBlur(kernel_size=3),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = self.img_labels.iloc[idx, 0]  # ชื่อไฟล์ของรูปภาพ
        label = self.img_labels.iloc[idx, 1]  # คลาสของรูปภาพ

        # แปลง label เป็น string เพื่อใช้กับ os.path.join()
        img_path = os.path.join(self.img_dir, str(label), img_name)

        # เปิดรูปภาพด้วย PIL
        image = Image.open(img_path).convert('RGB')

        # แปลง label จากชื่อคลาสเป็น index
        label = class_to_idx[str(label)]

        # ตรวจสอบว่าคลาสนี้มีตัวอย่างน้อยกว่าหรือไม่
        if self.class_sample_counts[label] < self.threshold:
            image = self.augment_transform(image)  # ทำ Data Augmentation เฉพาะกับคลาสที่มีข้อมูลน้อย
        else:
            if self.transform:
                image = self.transform(image)  # ใช้การแปลงปกติสำหรับคลาสที่มีข้อมูลมากกว่า threshold

        # ใช้ target_transform ถ้ามี
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

1. Preparing your data for evaluate with DataLoaders

In [6]:
import torchvision.transforms.v2 as transforms
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn


# Transformations for test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.ToImage(),  # Convert to image format
    transforms.ConvertImageDtype(torch.float32),  # Convert image dtype
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Load the test dataset
test_dataset = CustomImageDataset(annotations_file="./test_annotations.csv", img_dir="./thaicharecter", transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)



c:\Users\aumm\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [7]:
from torchvision.models import densenet121  # ใช้ DenseNet แทน
import torch.nn as nn

model = densenet121()
model.classifier = nn.Linear(1024, len(classes))  # ปรับจำนวนคลาสตาม dataset
model.load_state_dict(torch.load('./model.pt'))
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# ย้ายโมเดลไปยังอุปกรณ์ที่เหมาะสม (GPU หรือ CPU)
model.to(device)
model.eval()

# ประเมินผลโมเดล
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # ทำ forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # คำนวณจำนวนตัวอย่างทั้งหมดและตัวอย่างที่ทำนายถูกต้อง
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# คำนวณและแสดงค่า accuracy
accuracy = 100 * correct / total
print(f'Accuracy of the network on the test images: {accuracy:.02f} %')

Accuracy of the network on the test images: 98.85 %
